In [117]:
df[df.duplicated(['code','smiles'])]

,code,smiles,date,similarity,source,country,num,contries
20458,WO-2006049942-A9,CC(=O)Oc1ccccc1C(=O)O,2015-07-16,1.0,SureChEMBL,WO,365331,"[EP, JP, US, WO]"
259896,EP-2910589-B1,CC(=O)Oc1ccccc1C(=O)O,2019-09-11,1.0,SureChEMBL,EP,365331,"[EP, JP, US, WO]"


In [118]:
df[(df['code']=='WO-2006049942-A9')&(df['smiles']=='CC(=O)Oc1ccccc1C(=O)O')]

,code,smiles,date,similarity,source,country,num,contries
20457,WO-2006049942-A9,CC(=O)Oc1ccccc1C(=O)O,2015-04-09,1.0,SureChEMBL,WO,365331,"[EP, JP, US, WO]"
20458,WO-2006049942-A9,CC(=O)Oc1ccccc1C(=O)O,2015-07-16,1.0,SureChEMBL,WO,365331,"[EP, JP, US, WO]"


In [1]:
from rdkit import Chem
# from rdkit import DataStructs
# from rdkit import rdBase
# from rdkit.Chem.Draw import IPythonConsole
# from rdkit.Chem import Draw

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import json 
import pandas.io.sql as psql

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import psycopg2,time
# print(time.asctime(),"rdkit=",rdBase.rdkitVersion)

In [5]:
from rdkit.Chem import rdFMCS
# from rdkit.Chem.Draw import rdMolDraw2D
# from IPython.display import SVG

In [ ]:
user = # postgresql DB user ID
password = # password
host_product =  # 서버 IP 
dbname = # DB Name
port=# port number

In [7]:
product_connection_string = "dbname={dbname} user={user} host={host} password={password} port={port}".format(dbname=dbname,
                                    user=user,
                                    host=host_product,
                                    password=password,
                                    port=port) 

In [8]:
try:
    conn = psycopg2.connect(product_connection_string)
    print(f"Success to connect to the database '{dbname}'")
except:
    print("Unable to connect to the database")

Success to connect to the database 'patent'


In [9]:
inp = pd.read_csv('input.tsv',sep = '\t')
option = {"code":"JP,WO,EP,US","cutoff":0.8,"cpus":5,"num_save":500,"method":'substructure'}#"similarity"}

In [10]:
option

{'code': 'JP,WO,EP,US',
 'cutoff': 0.8,
 'cpus': 5,
 'num_save': 500,
 'method': 'substructure'}

In [11]:
inp

,id,smiles
0,Adavosertib,O=C1N(N(C2=NC(NC3=CC=C(C=C3)N4CCN(CC4)C)=NC=C2...
1,Afatinib,CN(C)CC=CC(=O)NC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC(...
2,Aspirin,CC(=O)OC1=CC=CC=C1C(=O)O
3,test,N1=CC=CN2N=CC(C(N)=O)=C12


In [12]:
name = inp['id'].tolist()
query = inp['smiles'].tolist()
threshold = option['cutoff']
num_input = len(name)
limit = option['num_save']
method = option['method']

In [13]:
query

['O=C1N(N(C2=NC(NC3=CC=C(C=C3)N4CCN(CC4)C)=NC=C21)C5=NC(C(O)(C)C)=CC=C5)CC=C',
 'CN(C)CC=CC(=O)NC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC(=C(C=C3)F)Cl)OC4CCOC4',
 'CC(=O)OC1=CC=CC=C1C(=O)O',
 'N1=CC=CN2N=CC(C(N)=O)=C12']

In [15]:
def get_SS_SMARTS(q,s):
    ms = Chem.MolFromSmiles(s)
    m = Chem.MolFromSmiles(q)
    mcs = rdFMCS.FindMCS([ms,m])
    
    return mcs.smartsString

In [19]:
output = pd.DataFrame(columns=  ['query_id','query_smiles','num_patent','num_comp','patent','result'])
start = time.time()
for i in range(num_input):
    a = time.time()
    if method == 'similarity':
        print('Start Similarity Search')
        print('start,',time.time())
        df = psql.read_sql(f"select db.code, db.smiles, db.date, res.similarity from db inner join \
                            (select * from get_mfp_neighbors('{query[i]}',{threshold},{int(limit)})) as res \
                             on db.smiles=res.smiles;",conn)
        df = df.sort_values('similarity',ascending = False).reset_index(drop = True)
    elif method == 'substructure':
        print('Start Substruture Search')
        print('start,',time.time())
        df = psql.read_sql(f"select db.code, db.smiles, db.date from db inner join \
                            (select smiles from mol_mfp where m@>cast(mol_to_smarts(mol_from_smiles('{query[i]}'))as text)::qmol limit {int(limit)}) as res \
                             on db.smiles=res.smiles;",conn)
    b = time.time()
    print('query end,',time.time())
    df['source'] = 'SureChEMBL' 
    df['country'] = df['code'].apply(lambda x:x[:2])
    df = df[df['country'].isin(option['code'].split(","))].reset_index(drop = True)
    print('country filtering,',time.time())

    num = df.groupby('smiles')['code'].nunique()
    df['num'] = df['smiles'].apply(lambda x: num.loc[x].item())
    kigdom = df.groupby('smiles')['country'].apply(lambda x: list(np.unique(x)))
    df['contries'] = df['smiles'].apply(lambda x: kigdom.loc[x])
    print('num contries groupby,',time.time())

    if method == 'similarity':
        df.columns = ['patent_code','smiles','date','similarity','source','each_country_code','num','country_code']
        df = df[['smiles','similarity','country_code','patent_code','date','source','num']]
        df = df.sort_values('similarity',ascending = False).reset_index(drop = True)
        print(time.time())

    elif method == 'substructure':
        df.columns = ['patent_code','smiles','date','source','each_country_code','num','country_code']
        df = df[['smiles','country_code','patent_code','date','source','num']]
        df = df.sort_values('num', ascending = False).reset_index(drop = True)
        print('sub sorting df,',time.time())
        
    code = pd.DataFrame(df.groupby('smiles')['patent_code'].apply(lambda x: list(np.unique(x)))).reset_index().to_dict(orient='record')
    
    print('unique patent code,',time.time())
    df = df.drop_duplicates('smiles',keep='first').reset_index(drop=True)
    df['method'] = option['method']
    print('drop duplicates,',time.time())
    df['smarts'] = df['smiles'].apply(lambda x: get_SS_SMARTS(query[i],x))        

    print('get MCS,',time.time())

    output.loc[i,'query_id'] = name[i]
    output.loc[i,'query_smiles'] = query[i]
    output.loc[i,'num_patent'] = num.sum()
    output.loc[i,'num_comp'] = df['smiles'].nunique()
    output.loc[i,'patent'] = code
    output.loc[i,'result'] = df.to_dict(orient = 'records')
    print('into output,',time.time())


    if i == 0: 
        print(f'>> {i+1}st compound done:')#, b-a)
    elif i == 1: 
        print(f'>> {i+1}nd compound done:')#, b-a)
    elif i == 2: 
        print(f'>> {i+1}rd compound done:')#, b-a)
    else:
        print(f'>> {i+1}th compound done:')#, b-a)

end = time.time()  
print(f'Searched patent for {round(end-start)} second')


Start Substruture Search
start, 1680679985.4012938
query end, 1680680015.640782
country filtering, 1680680015.6430674
num contries groupby, 1680680015.66061
sub sorting df, 1680680015.6613066
unique patent code, 1680680015.6653895
drop duplicates, 1680680015.665784
get MCS, 1680680015.811187
into output, 1680680015.8124967
>> 1st compound done:
Start Substruture Search
start, 1680680015.8125088
query end, 1680680045.7644854
country filtering, 1680680045.7691956
num contries groupby, 1680680045.841914
sub sorting df, 1680680045.8432472
unique patent code, 1680680045.849536
drop duplicates, 1680680045.8501427
get MCS, 1680680045.986864
into output, 1680680045.9883628
>> 2nd compound done:
Start Substruture Search
start, 1680680045.9883752
query end, 1680680076.9258926
country filtering, 1680680076.9280682
num contries groupby, 1680680077.0009823
sub sorting df, 1680680077.0021856
unique patent code, 1680680077.0164807
drop duplicates, 1680680077.0170734
get MCS, 1680680077.2162914
into o

In [252]:
output

,query_id,query_smiles,num_patent,num_comp,patent,result
0,Adavosertib,O=C1N(N(C2=NC(NC3=CC=C(C=C3)N4CCN(CC4)C)=NC=C2...,720,135,[{'smiles': 'C/C1=C(\C)Cn2c(=O)c3cnc(Nc4ccc(N5...,[{'smiles': 'C=CCn1c(=O)c2cnc(Nc3ccc(N4CCN(C)C...
1,Afatinib,CN(C)CC=CC(=O)NC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC(...,5719,149,[{'smiles': 'C#CCOCCCCCOc1ccc(-c2cc(Nc3ncnc4cc...,[{'smiles': 'CN(C)CC=CC(=O)Nc1cc2c(Nc3ccc(F)c(...
2,Aspirin,CC(=O)OC1=CC=CC=C1C(=O)O,4690,500,[{'smiles': 'C#COP(=O)(OC#C)OCCc1ccc(OC(=O)c2c...,"[{'smiles': 'CC(=O)Oc1cc(C(F)(F)F)ccc1C(=O)O',..."
3,test,N1=CC=CN2N=CC(C(N)=O)=C12,2900,500,[{'smiles': 'C#CC(NC(=O)c1cnn2c(C)cc(-c3cccc(C...,[{'smiles': 'COc1ncc(F)cc1C1CCCN1c1ccn2ncc(C(=...


In [251]:
output.result.iloc[0][0]

{'smiles': 'C=CCn1c(=O)c2cnc(Nc3ccc(N4CCN(C)CC4)cc3)nc2n1-c1cccc(C(C)(C)O)n1',
 'country_code': ['EP', 'US', 'WO'],
 'patent_code': 'WO-2011034743-A1',
 'date': '2011-03-24',
 'source': 'SureChEMBL',
 'num': 90,
 'method': 'substructure',
 'smarts': '[#6]=[#6]-[#6]-[#7]1:[#6](=[#8]):[#6]2:[#6](:[#7]:1-[#6]1:[#6]:[#6]:[#6]:[#6](:[#7]:1)-[#6](-[#6])(-[#6])-[#8]):[#7]:[#6](:[#7]:[#6]:2)-[#7]-[#6]1:[#6]:[#6]:[#6](:[#6]:[#6]:1)-[#7]1-[#6]-[#6]-[#7](-[#6]-[#6]-1)-[#6]'}

In [18]:
output2 = output.to_dict('list')
with open('sample_new_revised.json', 'w') as f:
    json.dump(output2, f)

In [93]:
df = psql.read_sql(f"select db.code, db.smiles, db.date, res.similarity from db inner join \
                            (select * from get_mfp_neighbors('{query[i]}',{threshold},{int(limit)})) as res \
                             on db.smiles=res.smiles;",conn)

In [94]:
df

,code,smiles,date,similarity
0,WO-2012129128-A1,CC(=O)Oc1ccccc1C(=O)O,2012-09-27,1.0
1,WO-2012129100-A1,CC(=O)Oc1ccccc1C(=O)O,2012-09-27,1.0
2,WO-2012129090-A1,CC(=O)Oc1ccccc1C(=O)O,2012-09-27,1.0
3,WO-2012129088-A1,CC(=O)Oc1ccccc1C(=O)O,2012-09-27,1.0
4,WO-2012129073-A2,CC(=O)Oc1ccccc1C(=O)O,2012-09-27,1.0
...,...,...,...,...
366018,WO-2012129491-A1,CC(=O)Oc1ccccc1C(=O)O,2012-09-27,1.0
366019,WO-2012129406-A1,CC(=O)Oc1ccccc1C(=O)O,2012-09-27,1.0
366020,WO-2012129347-A1,CC(=O)Oc1ccccc1C(=O)O,2012-09-27,1.0
366021,WO-2012129305-A1,CC(=O)Oc1ccccc1C(=O)O,2012-09-27,1.0


In [97]:
df['source'] = 'SureChEMBL' 
df['country'] = df['code'].apply(lambda x:x[:2])
df = df[df['country'].isin(option['code'].split(","))].reset_index(drop = True)

In [129]:
print(time.time())
num = df.groupby('smiles')['code'].nunique()
print(time.time())
df['num'] = df['smiles'].apply(lambda x: num.loc[x].item())
print(time.time())

1680504834.1490624
1680504834.3220537
1680504836.773932


In [107]:
print(time.time())
kigdom = df.groupby('smiles')['country'].apply(lambda x: list(np.unique(x)))
print(time.time())
df['contries'] = df['smiles'].apply(lambda x: kigdom.loc[x])
print(time.time())
df

1680501402.4519677
1680501402.721144
1680501404.8741457


,code,smiles,date,similarity,source,country,num,contries
0,WO-2012129128-A1,CC(=O)Oc1ccccc1C(=O)O,2012-09-27,1.0,SureChEMBL,WO,365331,"[EP, JP, US, WO]"
1,WO-2012129100-A1,CC(=O)Oc1ccccc1C(=O)O,2012-09-27,1.0,SureChEMBL,WO,365331,"[EP, JP, US, WO]"
2,WO-2012129090-A1,CC(=O)Oc1ccccc1C(=O)O,2012-09-27,1.0,SureChEMBL,WO,365331,"[EP, JP, US, WO]"
3,WO-2012129088-A1,CC(=O)Oc1ccccc1C(=O)O,2012-09-27,1.0,SureChEMBL,WO,365331,"[EP, JP, US, WO]"
4,WO-2012129073-A2,CC(=O)Oc1ccccc1C(=O)O,2012-09-27,1.0,SureChEMBL,WO,365331,"[EP, JP, US, WO]"
...,...,...,...,...,...,...,...,...
366018,WO-2012129491-A1,CC(=O)Oc1ccccc1C(=O)O,2012-09-27,1.0,SureChEMBL,WO,365331,"[EP, JP, US, WO]"
366019,WO-2012129406-A1,CC(=O)Oc1ccccc1C(=O)O,2012-09-27,1.0,SureChEMBL,WO,365331,"[EP, JP, US, WO]"
366020,WO-2012129347-A1,CC(=O)Oc1ccccc1C(=O)O,2012-09-27,1.0,SureChEMBL,WO,365331,"[EP, JP, US, WO]"
366021,WO-2012129305-A1,CC(=O)Oc1ccccc1C(=O)O,2012-09-27,1.0,SureChEMBL,WO,365331,"[EP, JP, US, WO]"


In [179]:
%timeit pd.DataFrame(res_temp.groupby('smiles')['code'].apply(lambda x: list(np.unique(x)))).reset_index().to_dict(orient='record') #code

646 µs ± 2.83 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [75]:
%timeit list(set(df['country']))

23.7 ms ± 314 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [76]:
%timeit df['country']df['country'].unique().tolist()

14 ms ± 33.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [80]:
%timeit ','.join(df['country'].unique().tolist())

14.1 ms ± 7.81 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [215]:
method = 'substructure'

In [10]:
# name = ['Aspirine']
# query = ['CC(=O)Oc1ccccc1C(=O)O']

In [17]:
output3 = pd.DataFrame(columns=  ['query_id','query_smiles','num_patent','num_comp','patent','result'])
start = time.time()
for i in range(len(name)):
    a = time.time()
    df = psql.read_sql(f"select db.code, db.smiles, db.date, res.similarity from db inner join (select * from get_mfp_neighbors('{query[i]}',{threshold},{int(limit)})) as res on db.smiles=res.smiles;",conn)
    df = df.sort_values('similarity',ascending = False).reset_index(drop = True)
    b = time.time()
    print(i, b-a)
    df['source'] = 'SureChEMBL' 
    df['country'] = df['code'].apply(lambda x:x[:2])
    df = df[df['country'].isin(option['code'].split(","))].reset_index(drop = True)

    num = df.groupby('smiles').count()['country']
    df['num'] = df['smiles'].apply(lambda x: num.loc[x].item())
    df['contries'] = df['country'].apply(lambda x: df['country'].unique().tolist())
    df.columns = ['patent_code','smiles','date','similarity','source','each_country_code','num','country_code']
    df = df[['smiles','similarity','country_code','patent_code','date','source','num']]
    

    code = df['patent_code'].unique().tolist()
    df = df.drop_duplicates('smiles',keep='first').reset_index(drop=True)
    df['method'] = option['method']

    output3.loc[i,'query_id'] = name[i]
    output3.loc[i,'query_smiles'] = query[i]
    output3.loc[i,'num_patent'] = len(code)
    output3.loc[i,'num_comp'] = df['smiles'].nunique()
    output3.loc[i,'patent'] = code
    output3.loc[i,'result'] = df.to_dict(orient = 'records')

output3

/tmp/ipykernel_3467325/2551370446.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = psql.read_sql(f"select db.code, db.smiles, db.date, res.similarity from db inner join (select * from get_mfp_neighbors('{query[i]}',{threshold},{int(limit)})) as res on db.smiles=res.smiles;",conn)


0 60.29944705963135


/tmp/ipykernel_3467325/2551370446.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = psql.read_sql(f"select db.code, db.smiles, db.date, res.similarity from db inner join (select * from get_mfp_neighbors('{query[i]}',{threshold},{int(limit)})) as res on db.smiles=res.smiles;",conn)


1 61.21188950538635


,query_id,query_smiles,num_patent,num_comp,patent,result
0,Adavosertib,O=C1N(N(C2=NC(NC3=CC=C(C=C3)N4CCN(CC4)C)=NC=C2...,90,2,"[US-20160289686-A1, WO-2014085216-A1, US-99447...",[{'smiles': 'C=CCn1c(=O)c2cnc(Nc3ccc(N4CCN(C)C...
1,Afatinib,CN(C)CC=CC(=O)NC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC(...,1131,7,"[US-20160287591-A1, WO-2014163714-A2, WO-20151...",[{'smiles': 'CN(C)C/C=C/C(=O)Nc1cc2c(Nc3ccc(F)...


In [20]:
df = psql.read_sql(f"select db.code, db.smiles, db.date, res.similarity from db inner join (select * from get_mfp_neighbors('{query[i]}',{threshold},{int(limit)})) as res on db.smiles=res.smiles;",conn)
df = df.sort_values('similarity',ascending = False).reset_index(drop = True)
b = time.time()
print(i, b-a)
df['source'] = 'SureChEMBL' 
df['country'] = df['code'].apply(lambda x:x[:2])
df = df[df['country'].isin(option['code'].split(","))].reset_index(drop = True)

num = df.groupby('smiles').count()['country']
df['num'] = df['smiles'].apply(lambda x: num.loc[x].item())
df['contries'] = df['country'].apply(lambda x: df['country'].unique().tolist())
df.columns = ['patent_code','smiles','date','similarity','source','each_country_code','num','country_code']
df = df[['smiles','similarity','country_code','patent_code','date','source','num']]

/tmp/ipykernel_3467325/3633533019.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = psql.read_sql(f"select db.code, db.smiles, db.date, res.similarity from db inner join (select * from get_mfp_neighbors('{query[i]}',{threshold},{int(limit)})) as res on db.smiles=res.smiles;",conn)


1 461.2774374485016


In [22]:
df['patent_code'].nunique()

1131

In [23]:
output3['result'].iloc[1]

[{'smiles': 'CN(C)C/C=C/C(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc1O[C@H]1CCOC1',
  'similarity': 1.0,
  'country_code': ['US', 'WO', 'EP', 'JP'],
  'patent_code': 'US-20160287591-A1',
  'date': '2016-10-06',
  'source': 'SureChEMBL',
  'num': 814,
  'method': 'Similarity'},
 {'smiles': 'CN(C)C/C=C/C(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc1OC1CCOC1',
  'similarity': 1.0,
  'country_code': ['US', 'WO', 'EP', 'JP'],
  'patent_code': 'WO-2014163714-A2',
  'date': '2014-10-09',
  'source': 'SureChEMBL',
  'num': 52,
  'method': 'Similarity'},
 {'smiles': 'CN(C)CC=CC(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc1OC1CCOC1',
  'similarity': 1.0,
  'country_code': ['US', 'WO', 'EP', 'JP'],
  'patent_code': 'WO-2015101554-A1',
  'date': '2015-07-09',
  'source': 'SureChEMBL',
  'num': 594,
  'method': 'Similarity'},
 {'smiles': 'CN(C)CC=CC(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc1O[C@H]1CCOC1',
  'similarity': 1.0,
  'country_code': ['US', 'WO', 'EP', 'JP'],
  'patent_code': 'WO-2015101554-A1',
  'date': '2015-07-09',

In [18]:
[x['num'] for x in output3['result'].iloc[0]]

[90, 8]

In [30]:
output3['result'].iloc[1]

[{'smiles': 'CN(C)CC=CC(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc1O[C@H]1CCOC1',
  'similarity': 1.0,
  'country_code': ['US', 'EP', 'WO', 'JP'],
  'patent_code': 'US-20190233433-A1',
  'date': '2019-08-01',
  'source': 'SureChEMBL',
  'num': 955,
  'method': 'Similarity'},
 {'smiles': 'CN(C)C/C=C/C(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc1O[C@H]1CCOC1',
  'similarity': 1.0,
  'country_code': ['US', 'EP', 'WO', 'JP'],
  'patent_code': 'EP-1478398-B1',
  'date': '2009-05-06',
  'source': 'SureChEMBL',
  'num': 814,
  'method': 'Similarity'},
 {'smiles': 'CN(C)CC=CC(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc1O[C@@H]1CCOC1',
  'similarity': 1.0,
  'country_code': ['US', 'EP', 'WO', 'JP'],
  'patent_code': 'EP-1478398-A1',
  'date': '2004-11-24',
  'source': 'SureChEMBL',
  'num': 648,
  'method': 'Similarity'},
 {'smiles': 'CN(C)CC=CC(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc1OC1CCOC1',
  'similarity': 1.0,
  'country_code': ['US', 'EP', 'WO', 'JP'],
  'patent_code': 'EP-1345910-B9',
  'date': '2014-06-04',
  's

In [22]:
output3['sim2'] = output3['similarity'].apply(lambda x: round(x,2))
output3['sim3'] = output3['similarity'].apply(lambda x: round(x,3))
output3['sim4'] = output3['similarity'].apply(lambda x: round(x,4))


KeyboardInterrupt



In [39]:
output2 = output3.to_dict('list')
with open('sample2.json', 'w') as f:
    json.dump(output2, f)

In [40]:
conn.close()